<a href="https://colab.research.google.com/github/nalivaikaanastasiya-dev/Agentic-RAG-Streamlit-App/blob/main/gent_rag_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# agent_rag_pipeline.ipynb
# Полноценный пример RAG-пайплайна

# 1. Загрузка переменных окружения
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CHROMA_DB_PATH = os.getenv("CHROMA_DB_PATH", "./chroma_db")

print("Переменные окружения загружены.")

# 2. Загрузка документов
from langchain.document_loaders import PyPDFLoader

# Замените 'example.pdf' на ваш файл
loader = PyPDFLoader("example.pdf")
docs = loader.load()
print(f"Загружено {len(docs)} документов.")

# 3. Разделение текста на чанки
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
print(f"Создано {len(chunks)} чанков текста.")

# 4. Создание векторного хранилища
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_DB_PATH)
vectordb.persist()
print("Векторное хранилище создано и сохранено.")

# 5. Создание RAG агента
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectordb.as_retriever())

# 6. Тестовый запрос
query = "Расскажи мне кратко про проект"
answer = qa.run(query)
print("Ответ агента:", answer)
